In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [3]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_qwen2 = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model2 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", device_map = 0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [4]:
def main_query_answering(query, max_tokens=100):
    """
    Answer the main query using Meta-Llama-3.2-3B and the retrieved documents.
    """
    # Prepare the prompt using the main query and the relevant shortened documents
    # Assuming 'body' key holds the document text in retrieved_documents
    prompt = f"You are an engine used to generate web search queries. Omit information you dont need, depending on what the user wants. Generate and return the correct search query. {query} \n. The web search query is (in 5 words or less):"

    # Tokenize input for LLM
    inputs = tokenizer_qwen2(prompt, return_tensors="pt").to("cuda")

    # Generate answer for the main query with reduced token output
    outputs = model2.generate(**inputs, max_new_tokens=max_tokens, temperature=0.5)

    # Decode and extract answer
    answer = tokenizer_qwen2.decode(outputs[0], skip_special_tokens=True).strip()

    return answer

In [5]:
query = "Show me tshirts with the same design"
desc = 'A man wears a black hoodie with the Superman logo.'
response = main_query_answering(f"The user wants: {query}. The given information is: {desc}.")
search_query = response.split('"')[1]
print(search_query)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


t-shirts with superman logo


Using the open source duckduckgo API to search and return results.

In [6]:
!pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.2 MB/s eta 0:00:00


In [7]:
from duckduckgo_search import DDGS


In [12]:
def websearch(query, desc):
    response = main_query_answering(f"The user wants: {query}. The given information is: {desc}.")
    search_query = response.split('"')[1]


    # results_text = DDGS().text(search_query, max_results=5)

    # results_img = DDGS().images(search_query, max_results=5)

    return search_query

In [9]:
!pip install fastapi uvicorn
!pip install fastapi uvicorn nest_asyncio pyngrok
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.6 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [13]:

from pyngrok import ngrok

# Set the Ngrok authentication token
ngrok.set_auth_token("2nRYGlbanqTVP1JoOsPlk8MKAWj_7hc88mo63ZNm1gCEosmh9")

# Open a tunnel to the FastAPI app running on port 8000
public_url = ngrok.connect(8000)

print(f"FastAPI is publicly available at: {public_url}")

FastAPI is publicly available at: NgrokTunnel: "https://e450-34-125-122-228.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

# Apply nest_asyncio to allow running uvicorn inside the notebook
nest_asyncio.apply()

# Define the FastAPI app
app = FastAPI()

# Define a Pydantic model for the input query
class QueryRequest(BaseModel):
    query: str

@app.post("/web_search")
async def web_search_endpoint(request: QueryRequest):
    query = request.query
    desc, final_query = query.split('||')
    final_answer = websearch(final_query, desc)  # Assuming rag_pipeline function is defined elsewhere
    return {"final_answer": final_answer}

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [1735]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2401:4900:5f30:64ed:4082:8723:6974:59da:0 - "POST /web_search HTTP/1.1" 200 OK
INFO:     2401:4900:5f30:64ed:4082:8723:6974:59da:0 - "POST /web_search HTTP/1.1" 200 OK
INFO:     2401:4900:5f30:64ed:4082:8723:6974:59da:0 - "POST /web_search HTTP/1.1" 200 OK
INFO:     2401:4900:5f30:64ed:4082:8723:6974:59da:0 - "POST /web_search HTTP/1.1" 200 OK
